In [24]:
import pandas as pd
import numpy as np

from nltk import pos_tag, word_tokenize
import re

import time
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import nltk
import nltk as nlp
import string
import re

In [10]:
train = pd.read_csv('Datasets\Single Source\Treated\combined_train_treated.csv')
test = pd.read_csv('Datasets\Single Source\Treated\combined_test_treated.csv')

In [11]:
train['text'].fillna("No Text", inplace = True)
test['text'].fillna("No Text", inplace = True)

train_copy = train.copy(deep = True)
test_copy = test.copy(deep = True)

## Logistic Regression

In [13]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 95.55%
CONFUSION MATRIX
[[3276  205]
 [  13 1403]]


## Support Vector Classifier

In [14]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 96.98%
CONFUSION MATRIX
[[3345  136]
 [  12 1404]]


## Multinomial Naive Bayes Classifier

In [15]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 96.98%
CONFUSION MATRIX
[[3345  136]
 [  12 1404]]


## Gradient Boost Classifier

In [16]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', GradientBoostingClassifier(loss = 'deviance',
                                                   learning_rate = 0.01,
                                                   n_estimators = 10,
                                                   max_depth = 5,
                                                   random_state=42))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))
print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 80.09%
CONFUSION MATRIX
[[2637  844]
 [ 131 1285]]


## XGBoost Classifier*

In [17]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', XGBClassifier(loss = 'deviance',
                                        learning_rate = 0.01,
                                        n_estimators = 10,
                                        max_depth = 5,
                                        random_state=2020))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))
print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 80.15%
CONFUSION MATRIX
[[2641  840]
 [ 132 1284]]


## Random Forest Classifier

In [18]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', RandomForestClassifier())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print("CONFUSION MATRIX")
print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 92.14%
CONFUSION MATRIX
[[3115  366]
 [  19 1397]]


# LSTM - RNN Baseline

In [19]:
X = train_copy['text']
Y = train_copy['target']

le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [20]:
max_words = 500
max_len = 1000

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X)

sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()

In [21]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),
             metrics = ['accuracy'])

In [22]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          25000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [25]:
#TAKES TIME. Fitting on GTX 1050ti.
t1 = time.time()

model.fit(sequences_matrix, Y, batch_size = 256, epochs = 20,
         validation_split = 0.2)

t2 = time.time()
print("Time Taken = ", t2-t1)

C:\Users\Siraz\Anaconda3\envs\gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 32000 samples, validate on 8000 samples
Epoch 1/20
32000/32000 [==============================] - 101s 3ms/step - loss: 0.2460 - accuracy: 0.9121 - val_loss: 0.1266 - val_accuracy: 0.9586
Epoch 2/20
32000/32000 [==============================] - 123s 4ms/step - loss: 0.1233 - accuracy: 0.9592 - val_loss: 0.0975 - val_accuracy: 0.9682
Epoch 3/20
32000/32000 [==============================] - 142s 4ms/step - loss: 0.1041 - accuracy: 0.9668 - val_loss: 0.0908 - val_accuracy: 0.9711
Epoch 4/20
32000/32000 [==============================] - 193s 6ms/step - loss: 0.0872 - accuracy: 0.9728 - val_loss: 0.0898 - val_accuracy: 0.9726
Epoch 5/20
32000/32000 [==============================] - 125s 4ms/step - loss: 0.0744 - accuracy: 0.9762 - val_loss: 0.0752 - val_accuracy: 0.9783
Epoch 6/20
32000/32000 [==============================] - 118s 4ms/step - loss: 0.0659 - accuracy: 0.9787 - val_loss: 0.0638 - val_accuracy: 0.9790
Epoch 7/20
32000/32000 [==============================] - 112s 

### LSTM on test data

In [26]:
test_sequences = tok.texts_to_sequences(test_copy['text'])
test_sequences_matrix = sequence.pad_sequences(test_sequences, 
                                               maxlen = max_len)

accr = model.evaluate(test_sequences_matrix, test_copy['target'])
print('Accuracy :{:0.2f}'.format(accr[1]))

4897/4897 [==============================] - 20s 4ms/step
Accuracy :0.88


In [28]:
#SAVING MODEL

# serialize model to JSON
model_json = model.to_json()
with open("ideal_LSTM_arc.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ideal_LSTM_weights.h5")